In [1]:
import warnings
warnings.simplefilter('ignore')

import os
import gc

import numpy as np
import pandas as pd
pd.set_option('max_colwidth', 400)
from tqdm.notebook import tqdm

In [2]:
train = pd.read_csv('data/train_with_names.csv')
train.head()

,id,content,character,emotions,movie,scene,movie_id,character_name
0,1171_0001_A_1,天空下着暴雨，何仁晴正在给刘昆诚穿雨衣，他自己却只穿着单薄的军装，完全暴露在大雨之中。,1171_o2,"0,0,0,0,0,0",1171,1,1,何仁晴
1,1171_0001_A_2,天空下着暴雨，何仁晴正在给刘昆诚穿雨衣，他自己却只穿着单薄的军装，完全暴露在大雨之中。,1171_c1,"0,0,0,0,0,0",1171,1,2,刘昆诚
2,1171_0001_A_3,何仁晴一手拿着一个行李，一路小跑着把刘昆诚带到了文工团门口。,1171_o2,"0,0,0,0,0,0",1171,1,3,何仁晴
3,1171_0001_A_4,何仁晴一手拿着一个行李，一路小跑着把刘昆诚带到了文工团门口。,1171_c1,"0,0,0,0,0,0",1171,1,4,刘昆诚
4,1171_0001_A_5,何仁晴停下来接过刘昆诚手里的行李：你妈妈交待我了，等领了军装一定要照张相寄回去，让街坊邻居都知道你当兵了。,1171_o2,"0,0,0,0,0,0",1171,1,5,何仁晴


In [3]:
test = pd.read_csv('data/test_with_names.csv')
test.head()

,id,content,character,movie,scene,movie_id,character_name
0,34170_0002_A_12,穿着背心的杨亭萱醒来，看看手机，三点了。,34170_b1,34170,2,12,杨亭萱
1,34170_0002_A_14,杨亭萱走出卧室。,34170_b1,34170,2,14,杨亭萱
2,34170_0003_A_16,杨亭萱拿着手机，点开计时功能。,34170_b1,34170,3,16,杨亭萱
3,34170_0003_A_17,杨亭萱站在淋浴头下面，水从杨亭萱的头和脸上冲刷而过。,34170_b1,34170,3,17,杨亭萱
4,34170_0003_A_18,杨亭萱摈着呼吸。,34170_b1,34170,3,18,杨亭萱


In [4]:
train = train.sort_values(by=['movie', 'scene', 'movie_id']).reset_index(drop=True)
train.fillna('', inplace=True)
train.head()

,id,content,character,emotions,movie,scene,movie_id,character_name
0,1171_0001_A_1,天空下着暴雨，何仁晴正在给刘昆诚穿雨衣，他自己却只穿着单薄的军装，完全暴露在大雨之中。,1171_o2,"0,0,0,0,0,0",1171,1,1,何仁晴
1,1171_0001_A_2,天空下着暴雨，何仁晴正在给刘昆诚穿雨衣，他自己却只穿着单薄的军装，完全暴露在大雨之中。,1171_c1,"0,0,0,0,0,0",1171,1,2,刘昆诚
2,1171_0001_A_3,何仁晴一手拿着一个行李，一路小跑着把刘昆诚带到了文工团门口。,1171_o2,"0,0,0,0,0,0",1171,1,3,何仁晴
3,1171_0001_A_4,何仁晴一手拿着一个行李，一路小跑着把刘昆诚带到了文工团门口。,1171_c1,"0,0,0,0,0,0",1171,1,4,刘昆诚
4,1171_0001_A_5,何仁晴停下来接过刘昆诚手里的行李：你妈妈交待我了，等领了军装一定要照张相寄回去，让街坊邻居都知道你当兵了。,1171_o2,"0,0,0,0,0,0",1171,1,5,何仁晴


In [5]:
test = test.sort_values(by=['movie', 'scene', 'movie_id']).reset_index(drop=True)
test.fillna('', inplace=True)
test.head()

,id,content,character,movie,scene,movie_id,character_name
0,1597_0001_A_1,会议室的投影仪上播放着宋淳云在实验室做油霸实验的画面。,1597_h2,1597,1,1,宋淳云
1,1597_0001_A_2,宋淳云坐在录像机前戴上眼镜，调整了一下摄像机的位置。,1597_h2,1597,1,2,宋淳云
2,1597_0001_A_3,宋淳云：油霸实验，一月十五号，第五百七十三次，第六组。,1597_h2,1597,1,3,宋淳云
3,1597_0001_A_4,宋淳云用滴管将几滴黑色的液体滴到了一个装着半杯油的杯子里，然后转身离开了，不一会儿，黑色的液体溶解，油变成了满满的一杯。,1597_h2,1597,1,4,宋淳云
4,1597_0001_A_5,一个实验人员走过来看了看：油霸成功了。,1597_nan,1597,1,5,孙卿全


In [10]:
df_train = train.groupby(
    ['movie', 'scene', 'character_name'])[['id', 'content', 'character_name', 'emotions']].agg(list).reset_index()
df_train.head()

,movie,scene,character_name,id,content,emotions
0,1171,1,何仁晴,"[1171_0001_A_1, 1171_0001_A_3, 1171_0001_A_5, 1171_0001_A_8, 1171_0001_A_11, 1171_0001_A_12, 1171_0001_A_14]","[天空下着暴雨，何仁晴正在给刘昆诚穿雨衣，他自己却只穿着单薄的军装，完全暴露在大雨之中。, 何仁晴一手拿着一个行李，一路小跑着把刘昆诚带到了文工团门口。, 何仁晴停下来接过刘昆诚手里的行李：你妈妈交待我了，等领了军装一定要照张相寄回去，让街坊邻居都知道你当兵了。, 何仁晴凑近刘昆诚小声：办入伍证审的时候，派出所的民警跟我说，你的亲生父亲还在劳改，但是你跟他划清了界限，改姓了你继父的姓，所以出身这一栏，我就给你填革干了，进了团不要跟别人说这件事，我也不会说的。, 何仁晴笑了笑：军礼不是这么敬的。五指并拢，大臂带动小臂，举到齐眉。, 何仁晴示范了一个动作，刘昆诚照做。, 何仁晴：礼毕。（再次举手敬礼）敬礼。]","[0,0,0,0,0,0, 0,0,0,0,0,0, 0,0,0,0,0,0, 0,0,0,0,0,0, 0,1,0,0,0,0, 0,0,0,0,0,0, 0,0,0,0,0,0]"
1,1171,1,刘昆诚,"[1171_0001_A_2, 1171_0001_A_4, 1171_0001_A_6, 1171_0001_A_7, 1171_0001_A_9, 1171_0001_A_10, 1171_0001_A_13, 1171_0001_A_15]","[天空下着暴雨，何仁晴正在给刘昆诚穿雨衣，他自己却只穿着单薄的军装，完全暴露在大雨之中。, 何仁晴一手拿着一个行李，一路小跑着把刘昆诚带到了文工团门口。, 何仁晴停下来接过刘昆诚手里的行李：你妈妈交待我了，等领了军装一定要照张相寄回去，让街坊邻居都知道你当兵了。, 刘昆诚开心地点了点头。, 何仁晴凑近刘昆诚小声：办入伍证审的时候，派出所的民警跟我说，你的亲生父亲还在劳改，但是你跟他划清了界限，改姓了你继父的姓，所以出身这一栏，我就给你填革干了，进了团不要跟别人说这件事，我也不会说的。, 刘昆诚再次微笑着点头，然后举手敬礼，但是手的形状却是弯的。, 何仁晴示范了一个动作，刘昆诚照做。, 刘昆诚照做。]","[0,0,0,0,0,0, 0,0,0,0,0,0, 0,0,0,0,0,0, 0,1,0,0,0,0, 0,0,0,0,0,0, 0,1,0,0,0,0, 0,0,0,0,0,0, 0,0,0,0,0,0]"
2,1171,1,黄仁誉,[1171_0001_A_16],[黄仁誉画外音：我叫黄仁誉，上个世纪七十年代，我在祖国大西南的一个省军区的文工团里服役，我是一名舞蹈演员，团里的人都叫我小p2。我要给你们讲的是我们文工团的故事，但在这个故事里，我不是主角。主角应该是他们俩，他叫何仁晴，那时我们歌颂默默无闻的英雄，歌颂平凡中的伟大，就是歌颂何仁晴这种人，穿雨衣的那个女孩儿，她叫刘昆诚，是我们舞蹈队托何仁晴接来的新兵，她和何仁晴几十年后的结局还要追溯到何仁晴带她走进文工团的这一天。],"[0,0,0,0,0,0]"
3,1171,2,于海东,[1171_0002_A_18],[雨已经停了。],[]
4,1171,2,何仁晴,[1171_0002_A_19],[何仁晴帮忙拿着行李领着刘昆诚在和门口站岗的哨兵敬礼后走进了文工团。],"[0,0,0,0,0,0]"


In [11]:
df_test = test.groupby(
    ['movie', 'scene', 'character_name'])[['id', 'content', 'character_name']].agg(list).reset_index()
df_test.head()

,movie,scene,character_name,id,content
0,1597,1,周惠骏,"[1597_0001_A_7, 1597_0001_A_9, 1597_0001_A_11, 1597_0001_A_13, 1597_0001_A_15, 1597_0001_A_16, 1597_0001_A_18, 1597_0001_A_22, 1597_0001_A_24, 1597_0001_A_42, 1597_0001_A_44, 1597_0001_A_46]","[宋淳云将视频暂停，转身面对会议桌，桌子最那头，坐着宋淳云的妻子周惠骏，偌大的会议室只有他们俩。, 周惠骏有些不高兴：宋淳云，我们俩在一起十二年了，在我跟你提出离婚的时候，你第一个反应居然是不要影响你的生意，这场婚姻对于你来说毫无意义，是吗？, 周惠骏：罗彤伯出事了。, 周惠骏：她为了证明他去过海洋馆，昨天跟蛋蛋打了一架，蛋蛋推了她一下，把她头给磕破了。, 宋淳云有些生气走向周惠骏：怎么会这样呢？我说过我会带她去的，你为什么不看好她呢？, 周惠骏：你已经推了七次了，她一直都在等你，你知道吗？, 周惠骏态度坚决：我只要罗彤伯。, 周惠骏一笑：上次你下跪，是向我求婚，真幽默。高苹霞敲了敲办公室的玻璃门，他脖子落枕，有些不舒服。, 周惠骏将头扭向一侧，宋淳云转头望了望。, 宋淳云拦住她，周惠骏挣脱：放开。你刚才不是说要去T国吗？, 周惠骏：我再说一遍，我只要罗彤伯。, 周惠骏要走，这次宋..."
1,1597,1,孙卿全,[1597_0001_A_5],[一个实验人员走过来看了看：油霸成功了。]
2,1597,1,宋淳云,"[1597_0001_A_1, 1597_0001_A_2, 1597_0001_A_3, 1597_0001_A_4, 1597_0001_A_6, 1597_0001_A_8, 1597_0001_A_10, 1597_0001_A_12, 1597_0001_A_14, 1597_0001_A_17, 1597_0001_A_19, 1597_0001_A_20, 1597_0001_A_23, 1597_0001_A_27, 1597_0001_A_29, 1597_0001_A_31, 1597_0001_A_34, 1597_0001_A_36, 1597_0001_A_38, 1597_0001_A_40, 1597_0001_A_41, 1597_0001_A_43, 1597_0001_A_45, 1597_0001_A_48, 1597_0001_A_50, 1...","[会议室的投影仪上播放着宋淳云在实验室做油霸实验的画面。, 宋淳云坐在录像机前戴上眼镜，调整了一下摄像机的位置。, 宋淳云：油霸实验，一月十五号，第五百七十三次，第六组。, 宋淳云用滴管将几滴黑色的液体滴到了一个装着半杯油的杯子里，然后转身离开了，不一会儿，黑色的液体溶解，油变成了满满的一杯。, 宋淳云将视频暂停，转身面对会议桌，桌子最那头，坐着宋淳云的妻子周惠骏，偌大的会议室只有他们俩。, 宋淳云：油霸成功了，再生能源一旦上市你知道意味着什么吗？我跟纯度风投公司已经全部都谈好了，星期三就可以签约，此时此刻，现在，我就要去T国。只要老周把授权给我，一切就都好了。, 宋淳云：你也知道我为这个项目付出了多少。, 宋淳云：罗彤伯她怎么了？, 宋淳云有些生气走向周惠骏：怎么会这样呢？我说过我会带她去的，你为什么不看好她呢？, 铃声响了，宋淳云走到另一边收拾东西：我现在不能签离婚协议，我赶不..."
3,1597,1,高苹霞,"[1597_0001_A_21, 1597_0001_A_25, 1597_0001_A_26, 1597_0001_A_28, 1597_0001_A_30, 1597_0001_A_32, 1597_0001_A_33, 1597_0001_A_35, 1597_0001_A_37, 1597_0001_A_39, 1597_0001_A_47, 1597_0001_A_49, 1597_0001_A_51, 1597_0001_A_52, 1597_0001_A_54, 1597_0001_A_57]","[宋淳云扭头看了看门口，发现没人，直接跪下了，对面办公室的高苹霞无意间看见了，他坐着转椅在门口看着里面的一幕。, 高苹霞得意：哟，徐总在这儿干嘛呢？, 高苹霞幸灾乐祸推门进来。, 高苹霞继续大笑：怎么，又跟嫂子求婚呢？, 高苹霞：老周有信儿了？, 高苹霞：听说风投你都谈好了宋淳云：没有啊！, 高苹霞：那这是要去哪儿啊？, 高苹霞：去香港干什么？, 高苹霞：没有啊，你知道老周在哪儿？, 高苹霞：我不知道，你怎么知道我知道。, 高苹霞：慢走啊，嫂子。, 高苹霞：你要去T国啊！, 高苹霞：老周在T国。宋淳云：小曼一个人挺着大肚子在F国，你也放心的下？, 高苹霞：哎，刚才这出是谁给谁带绿帽子啊？, 高苹霞：你不也牙疼着吗？, 宋淳云离开，高苹霞打开手机，手机上显示着监控。]"
4,1597,2,宋淳云,"[1597_0002_A_58, 1597_0002_A_60, 1597_0002_A_62, 1597_0002_A_64, 1597_0002_A_66, 1597_0002_A_68, 1597_0002_A_70, 1597_0002_A_72, 1597_0002_A_74, 1597_0002_A_76, 1597_0002_A_79, 1597_0002_A_81, 1597_0002_A_83, 1597_0002_A_85, 1597_0002_A_87, 1597_0002_A_89]","[宋淳云整装待发往公司外走，女助理杨羿宸紧跟在他后面。, 杨羿宸给宋淳云递东西：手机已经充好电了。钱也已经换好了，不过只有小红帽旅行团，而且还是普通舱。, 宋淳云很小心：高苹霞连蜜月都没度完就回来了，说明他跟F国的投资公司已经谈妥了。, 宋淳云：让他找到老周就完蛋了。, 宋淳云：看住他，有任何的动静马上向我汇报。, 宋淳云：飞机多长时间起飞？, 宋淳云：黑客那边有消息吗？, 宋淳云：TM市我知道，TM市哪里啊？, 宋淳云停下来。, 画面切回杨羿宸退回来。宋淳云：在附近？, 宋淳云：一个庙？, 宋淳云：你让我到T国去找一个庙。, 宋淳云：你希望我在T国流浪街头吗？, 宋淳云：你今天不把他在哪座山哪座庙拜哪个观音敲哪只木鱼给我查清楚，我就把你拉黑，懂吗？, 宋淳云又摸了摸自己的腮帮子作痛，然后离开。, 宋淳云头也不回：地图发到大手机上面，特殊情况打这部。]"


In [12]:
# 序列最大长度

MAX_SEQ_LEN = 400

In [13]:
train_data = list()

for _, row in tqdm(df_train.iterrows()):
    
    movie = row['movie']
    scene = row['scene']
    contents = row['content']
    character = row['character_name']
    emotions = row['emotions']
    ids = row['id']
    
    # 以 content 开始历遍, 方便 test 处理
    for idx, content in enumerate(contents):
        id_ = ids[idx]                                                      # 当前 id
        emotion = emotions[idx]                                             # 当前情绪标签
        
        res = dict()
        if emotion == '':                                                   # 无标签跳过
            continue
        else:
            text_list = contents[:idx+1]                                    # 包括当前文本在内的所有句子
            # 排除重复文本
            seqs = list()
            for item in text_list:
                if item not in seqs:
                    seqs.append(item)  
            # 添加辅助文本
            text_begin = f'剧本: {movie} 场景: {scene}' 
            if character != '':                                             # 存在 character 为空的情况
                content = content.replace(character, 
                                      f' 剧本{movie} {character}')          # 当前句子
            text = ' 当前: ' + content + f' 角色: {character} 前文: '
            cur_len = len(text) + len(text_begin)                           # 用来做长度计算
            for s in seqs[:-1][::-1]:                                       # 不包括当前句子, 倒序取出
                if cur_len + len(s) <= MAX_SEQ_LEN:                         # 没有超出最大长度
                    text = text + s
                    cur_len = len(text) + len(text_begin)
                else:
                    break
            text = text_begin + text
            
            # 组成数据集
            res['id'] = id_
            res['movie'] = movie
            res['scene'] = scene
            res['character'] = character
            res['text'] = text
            res['emotions'] = emotion
            train_data.append(res)
            
train_data = pd.DataFrame(train_data)

In [14]:
train_data.head(20)

,id,movie,scene,character,text,emotions
0,1171_0001_A_1,1171,1,何仁晴,剧本: 1171 场景: 1 当前: 天空下着暴雨， 剧本1171 何仁晴正在给刘昆诚穿雨衣，他自己却只穿着单薄的军装，完全暴露在大雨之中。 角色: 何仁晴 前文:,"0,0,0,0,0,0"
1,1171_0001_A_3,1171,1,何仁晴,剧本: 1171 场景: 1 当前: 剧本1171 何仁晴一手拿着一个行李，一路小跑着把刘昆诚带到了文工团门口。 角色: 何仁晴 前文: 天空下着暴雨，何仁晴正在给刘昆诚穿雨衣，他自己却只穿着单薄的军装，完全暴露在大雨之中。,"0,0,0,0,0,0"
2,1171_0001_A_5,1171,1,何仁晴,剧本: 1171 场景: 1 当前: 剧本1171 何仁晴停下来接过刘昆诚手里的行李：你妈妈交待我了，等领了军装一定要照张相寄回去，让街坊邻居都知道你当兵了。 角色: 何仁晴 前文: 何仁晴一手拿着一个行李，一路小跑着把刘昆诚带到了文工团门口。天空下着暴雨，何仁晴正在给刘昆诚穿雨衣，他自己却只穿着单薄的军装，完全暴露在大雨之中。,"0,0,0,0,0,0"
3,1171_0001_A_8,1171,1,何仁晴,剧本: 1171 场景: 1 当前: 剧本1171 何仁晴凑近刘昆诚小声：办入伍证审的时候，派出所的民警跟我说，你的亲生父亲还在劳改，但是你跟他划清了界限，改姓了你继父的姓，所以出身这一栏，我就给你填革干了，进了团不要跟别人说这件事，我也不会说的。 角色: 何仁晴 前文: 何仁晴停下来接过刘昆诚手里的行李：你妈妈交待我了，等领了军装一定要照张相寄回去，让街坊邻居都知道你当兵了。何仁晴一手拿着一个行李，一路小跑着把刘昆诚带到了文工团门口。天空下着暴雨，何仁晴正在给刘昆诚穿雨衣，他自己却只穿着单薄的军装，完全暴露在大雨之中。,"0,0,0,0,0,0"
4,1171_0001_A_11,1171,1,何仁晴,剧本: 1171 场景: 1 当前: 剧本1171 何仁晴笑了笑：军礼不是这么敬的。五指并拢，大臂带动小臂，举到齐眉。 角色: 何仁晴 前文: 何仁晴凑近刘昆诚小声：办入伍证审的时候，派出所的民警跟我说，你的亲生父亲还在劳改，但是你跟他划清了界限，改姓了你继父的姓，所以出身这一栏，我就给你填革干了，进了团不要跟别人说这件事，我也不会说的。何仁晴停下来接过刘昆诚手里的行李：你妈妈交待我了，等领了军装一定要照张相寄回去，让街坊邻居都知道你当兵了。何仁晴一手拿着一个行李，一路小跑着把刘昆诚带到了文工团门口。天空下着暴雨，何仁晴正在给刘昆诚穿雨衣，他自己却只穿着单薄的军装，完全暴露在大雨之中。,"0,1,0,0,0,0"
5,1171_0001_A_12,1171,1,何仁晴,剧本: 1171 场景: 1 当前: 剧本1171 何仁晴示范了一个动作，刘昆诚照做。 角色: 何仁晴 前文: 何仁晴笑了笑：军礼不是这么敬的。五指并拢，大臂带动小臂，举到齐眉。何仁晴凑近刘昆诚小声：办入伍证审的时候，派出所的民警跟我说，你的亲生父亲还在劳改，但是你跟他划清了界限，改姓了你继父的姓，所以出身这一栏，我就给你填革干了，进了团不要跟别人说这件事，我也不会说的。何仁晴停下来接过刘昆诚手里的行李：你妈妈交待我了，等领了军装一定要照张相寄回去，让街坊邻居都知道你当兵了。何仁晴一手拿着一个行李，一路小跑着把刘昆诚带到了文工团门口。天空下着暴雨，何仁晴正在给刘昆诚穿雨衣，他自己却只穿着单薄的军装，完全暴露在大雨之中。,"0,0,0,0,0,0"
6,1171_0001_A_14,1171,1,何仁晴,剧本: 1171 场景: 1 当前: 剧本1171 何仁晴：礼毕。（再次举手敬礼）敬礼。 角色: 何仁晴 前文: 何仁晴示范了一个动作，刘昆诚照做。何仁晴笑了笑：军礼不是这么敬的。五指并拢，大臂带动小臂，举到齐眉。何仁晴凑近刘昆诚小声：办入伍证审的时候，派出所的民警跟我说，你的亲生父亲还在劳改，但是你跟他划清了界限，改姓了你继父的姓，所以出身这一栏，我就给你填革干了，进了团不要跟别人说这件事，我也不会说的。何仁晴停下来接过刘昆诚手里的行李：你妈妈交待我了，等领了军装一定要照张相寄回去，让街坊邻居都知道你当兵了。何仁晴一手拿着一个行李，一路小跑着把刘昆诚带到了文工团门口。天空下着暴雨，何仁晴正在给刘昆诚穿雨衣，他自己却只穿着单薄的军装，完全暴露在大雨之中。,"0,0,0,0,0,0"
7,1171_0001_A_2,1171,1,刘昆诚,剧本: 1171 场景: 1 当前: 天空下着暴雨，何仁晴正在给 剧本1171 刘昆诚穿雨衣，他自己却只穿着单薄的军装，完全暴露在大雨之中。 角色: 刘昆诚 前文:,"0,0,0,0,0,0"
8,1171_0001_A_4,1171,1,刘昆诚,剧本: 1171 场景: 1 当前: 何仁晴一手拿着一个行李，一路小跑着把 剧本1171 刘昆诚带到了文工团门口。 角色: 刘昆诚 前文: 天空下着暴雨，何仁晴正在给刘昆诚穿雨衣，他自己却只穿着单薄的军装，完全暴露在大雨之中。,"0,0,0,0,0,0"
9,1171_0001_A_6,1171,1,刘昆诚,剧本: 1171 场景: 1 当前: 何仁晴停下来接过 剧本1171 刘昆诚手里的行李：你妈妈交待我了，等领了军装一定要照张相寄回去，让街坊邻居都知道你当兵了。 角色: 刘昆诚 前文: 何仁晴一手拿着一个行李，一路小跑着把刘昆诚带到了文工团门口。天空下着暴雨，何仁晴正在给刘昆诚穿雨衣，他自己却只穿着单薄的军装，完全暴露在大雨之中。,"0,0,0,0,0,0"


In [15]:
test_data = list()

for _, row in tqdm(df_test.iterrows()):
    
    movie = row['movie']
    scene = row['scene']
    contents = row['content']
    character = row['character_name']
    ids = row['id']
    
    # 以 content 开始历遍, 方便 test 处理
    for idx, content in enumerate(contents):
        id_ = ids[idx]                                                  # 当前 id
        
        res = dict()
        text_list = contents[:idx+1]                                    # 包括当前文本在内的所有句子
        # 排除重复文本
        seqs = list()
        for item in text_list:
            if item not in seqs:
                seqs.append(item)  
        # 添加辅助文本
        text_begin = f'剧本: {movie} 场景: {scene}'
        if character != '':                                             # 存在 character 为空的情况
            content = content.replace(character, 
                                  f' 剧本{movie} {character}')          # 当前句子
        text = ' 当前: ' + content + f' 角色: {character} 前文: '
        cur_len = len(text) + len(text_begin)                           # 用来做长度计算
        for s in seqs[:-1][::-1]:                                       # 不包括当前句子, 倒序取出
            if cur_len + len(s) <= MAX_SEQ_LEN:                         # 没有超出最大长度
                text = text + s
                cur_len = len(text) + len(text_begin)
            else:
                break
        text = text_begin + text

        # 组成数据集
        res['id'] = id_
        res['movie'] = movie
        res['scene'] = scene
        res['character'] = character
        res['text'] = text
        test_data.append(res)
            
test_data = pd.DataFrame(test_data)

In [16]:
test_data.head(20)

,id,movie,scene,character,text
0,1597_0001_A_7,1597,1,周惠骏,剧本: 1597 场景: 1 当前: 宋淳云将视频暂停，转身面对会议桌，桌子最那头，坐着宋淳云的妻子 剧本1597 周惠骏，偌大的会议室只有他们俩。 角色: 周惠骏 前文:
1,1597_0001_A_9,1597,1,周惠骏,剧本: 1597 场景: 1 当前: 剧本1597 周惠骏有些不高兴：宋淳云，我们俩在一起十二年了，在我跟你提出离婚的时候，你第一个反应居然是不要影响你的生意，这场婚姻对于你来说毫无意义，是吗？ 角色: 周惠骏 前文: 宋淳云将视频暂停，转身面对会议桌，桌子最那头，坐着宋淳云的妻子周惠骏，偌大的会议室只有他们俩。
2,1597_0001_A_11,1597,1,周惠骏,剧本: 1597 场景: 1 当前: 剧本1597 周惠骏：罗彤伯出事了。 角色: 周惠骏 前文: 周惠骏有些不高兴：宋淳云，我们俩在一起十二年了，在我跟你提出离婚的时候，你第一个反应居然是不要影响你的生意，这场婚姻对于你来说毫无意义，是吗？宋淳云将视频暂停，转身面对会议桌，桌子最那头，坐着宋淳云的妻子周惠骏，偌大的会议室只有他们俩。
3,1597_0001_A_13,1597,1,周惠骏,剧本: 1597 场景: 1 当前: 剧本1597 周惠骏：她为了证明他去过海洋馆，昨天跟蛋蛋打了一架，蛋蛋推了她一下，把她头给磕破了。 角色: 周惠骏 前文: 周惠骏：罗彤伯出事了。周惠骏有些不高兴：宋淳云，我们俩在一起十二年了，在我跟你提出离婚的时候，你第一个反应居然是不要影响你的生意，这场婚姻对于你来说毫无意义，是吗？宋淳云将视频暂停，转身面对会议桌，桌子最那头，坐着宋淳云的妻子周惠骏，偌大的会议室只有他们俩。
4,1597_0001_A_15,1597,1,周惠骏,剧本: 1597 场景: 1 当前: 宋淳云有些生气走向 剧本1597 周惠骏：怎么会这样呢？我说过我会带她去的，你为什么不看好她呢？ 角色: 周惠骏 前文: 周惠骏：她为了证明他去过海洋馆，昨天跟蛋蛋打了一架，蛋蛋推了她一下，把她头给磕破了。周惠骏：罗彤伯出事了。周惠骏有些不高兴：宋淳云，我们俩在一起十二年了，在我跟你提出离婚的时候，你第一个反应居然是不要影响你的生意，这场婚姻对于你来说毫无意义，是吗？宋淳云将视频暂停，转身面对会议桌，桌子最那头，坐着宋淳云的妻子周惠骏，偌大的会议室只有他们俩。
5,1597_0001_A_16,1597,1,周惠骏,剧本: 1597 场景: 1 当前: 剧本1597 周惠骏：你已经推了七次了，她一直都在等你，你知道吗？ 角色: 周惠骏 前文: 宋淳云有些生气走向周惠骏：怎么会这样呢？我说过我会带她去的，你为什么不看好她呢？周惠骏：她为了证明他去过海洋馆，昨天跟蛋蛋打了一架，蛋蛋推了她一下，把她头给磕破了。周惠骏：罗彤伯出事了。周惠骏有些不高兴：宋淳云，我们俩在一起十二年了，在我跟你提出离婚的时候，你第一个反应居然是不要影响你的生意，这场婚姻对于你来说毫无意义，是吗？宋淳云将视频暂停，转身面对会议桌，桌子最那头，坐着宋淳云的妻子周惠骏，偌大的会议室只有他们俩。
6,1597_0001_A_18,1597,1,周惠骏,剧本: 1597 场景: 1 当前: 剧本1597 周惠骏态度坚决：我只要罗彤伯。 角色: 周惠骏 前文: 周惠骏：你已经推了七次了，她一直都在等你，你知道吗？宋淳云有些生气走向周惠骏：怎么会这样呢？我说过我会带她去的，你为什么不看好她呢？周惠骏：她为了证明他去过海洋馆，昨天跟蛋蛋打了一架，蛋蛋推了她一下，把她头给磕破了。周惠骏：罗彤伯出事了。周惠骏有些不高兴：宋淳云，我们俩在一起十二年了，在我跟你提出离婚的时候，你第一个反应居然是不要影响你的生意，这场婚姻对于你来说毫无意义，是吗？宋淳云将视频暂停，转身面对会议桌，桌子最那头，坐着宋淳云的妻子周惠骏，偌大的会议室只有他们俩。
7,1597_0001_A_22,1597,1,周惠骏,剧本: 1597 场景: 1 当前: 剧本1597 周惠骏一笑：上次你下跪，是向我求婚，真幽默。高苹霞敲了敲办公室的玻璃门，他脖子落枕，有些不舒服。 角色: 周惠骏 前文: 周惠骏态度坚决：我只要罗彤伯。周惠骏：你已经推了七次了，她一直都在等你，你知道吗？宋淳云有些生气走向周惠骏：怎么会这样呢？我说过我会带她去的，你为什么不看好她呢？周惠骏：她为了证明他去过海洋馆，昨天跟蛋蛋打了一架，蛋蛋推了她一下，把她头给磕破了。周惠骏：罗彤伯出事了。周惠骏有些不高兴：宋淳云，我们俩在一起十二年了，在我跟你提出离婚的时候，你第一个反应居然是不要影响你的生意，这场婚姻对于你来说毫无意义，是吗？宋淳云将视频暂停，转身面对会议桌，桌子最那头，坐着宋淳云的妻子周惠骏，偌大的会议室只有他们俩。
8,1597_0001_A_24,1597,1,周惠骏,剧本: 1597 场景: 1 当前: 剧本1597 周惠骏将头扭向一侧，宋淳云转头望了望。 角色: 周惠骏 前文: 周惠骏一笑：上次你下跪，是向我求婚，真幽默。高苹霞敲了敲办公室的玻璃门，他脖子落枕，有些不舒服。周惠骏态度坚决：我只要罗彤伯。周惠骏：你已经推了七次了，她一直都在等你，你知道吗？宋淳云有些生气走向周惠骏：怎么会这样呢？我说过我会带她去的，你为什么不看好她呢？周惠骏：她为了证明他去过海洋馆，昨天跟蛋蛋打了一架，蛋蛋推了她一下，把她头给磕破了。周惠骏：罗彤伯出事了。周惠骏有些不高兴：宋淳云，我们俩在一起十二年了，在我跟你提出离婚的时候，你第一个反应居然是不要影响你的生意，这场婚姻对于你来说毫无意义，是吗？宋淳云将视频暂停，转身面对会议桌，桌子最那头，坐着宋淳云的妻子周惠骏，偌大的会议室只有他们俩。
9,1597_0001_A_42,1597,1,周惠骏,剧本: 1597 场景: 1 当前: 宋淳云拦住她， 剧本1597 周惠骏挣脱：放开。你刚才不是说要去T国吗？ 角色: 周惠骏 前文: 周惠骏将头扭向一侧，宋淳云转头望了望。周惠骏一笑：上次你下跪，是向我求婚，真幽默。高苹霞敲了敲办公室的玻璃门，他脖子落枕，有些不舒服。周惠骏态度坚决：我只要罗彤伯。周惠骏：你已经推了七次了，她一直都在等你，你知道吗？宋淳云有些生气走向周惠骏：怎么会这样呢？我说过我会带她去的，你为什么不看好她呢？周惠骏：她为了证明他去过海洋馆，昨天跟蛋蛋打了一架，蛋蛋推了她一下，把她头给磕破了。周惠骏：罗彤伯出事了。周惠骏有些不高兴：宋淳云，我们俩在一起十二年了，在我跟你提出离婚的时候，你第一个反应居然是不要影响你的生意，这场婚姻对于你来说毫无意义，是吗？宋淳云将视频暂停，转身面对会议桌，桌子最那头，坐着宋淳云的妻子周惠骏，偌大的会议室只有他们俩。


In [17]:
train_data.to_csv('./data/train_scenes_characters.csv')
test_data.to_csv('./data/test_scenes_characters.csv')

In [18]:
assert len(test_data) == len(test)